# Scrap http://wenshu.court.gov.cn/

In [75]:
import time
import numpy as np
import pandas as pd
from selenium import webdriver

browser = webdriver.Chrome('../../../../chromedriver')

In [76]:
browser.get('http://wenshu.court.gov.cn/')

In [79]:
browser.find_element_by_css_selector("input#head_maxsearch_btn").click()
browser.find_element_by_css_selector("input#completxt").send_keys('穿山甲')
#select 刑事案件
browser.find_element_by_css_selector("input#selectTree2_input").click()
browser.find_element_by_css_selector("a#selectTree2_tree_2_a").click()
browser.find_element_by_css_selector("input#list_btnmaxsearch").click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"input#head_maxsearch_btn"}
  (Session info: chrome=74.0.3729.169)
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Mac OS X 10.14.0 x86_64)


In [78]:
def extractPage():
    page = list()
    items = browser.find_elements_by_css_selector(".dataItem")
    for item in items:
        labels = item.find_elements_by_css_selector("div.label div.ajlx_lable")
        stage = ' '.join(str(x.text) for x in labels)
        key = item.get_attribute('key')
        realid = browser.execute_script('return com.str.Decrypt(unzip(arguments[0]))', key)
        realurl = 'http://wenshu.court.gov.cn/content/content?DocID=' + realid + '&KeyWord=' + '穿山甲'
        page.append({
            'judgedate': item.get_attribute('judgedate'),
            'casenumber': item.get_attribute('casenumber'),
            'casecourt': item.get_attribute('casecourt'),
            'stage': stage,
            'title': item.get_attribute('title'),
            'realid': realid,
            'url': realurl
        })
    time.sleep(1)
    return page

In [80]:
pages = list()

In [85]:
new = extractPage()
for item in new:
    pages.append(item)

In [86]:
len(pages)

20

In [87]:
df = pd.DataFrame(pages)

In [88]:
years = df['casenumber'].str.extract(r'.*（(.*)）.*', expand=False)
df['year'] = years
df['prosecution'], df['stage'] = df['stage'].str.split(' ', 1).str
df.head()

,casecourt,casenumber,judgedate,realid,stage,title,url,year,prosecution
0,瑞丽市人民法院,（2018）云3102刑初43号,2018-04-27,43284650-f27b-407b-8a9c-a90d00acd80c,一审,王军亿走私国家禁止进出口的货物、物品一审刑事判决书,http://wenshu.court.gov.cn/content/content?Doc...,2018,刑事
1,肥城市人民法院,（2018）鲁0983刑初160号,2018-10-25,189b4394-d4c2-4da6-8e24-a998017ea3c5,一审,汤占朴非法吸收公众存款一审刑事判决书,http://wenshu.court.gov.cn/content/content?Doc...,2018,刑事
2,邵阳市北塔区人民法院,（2013）北刑初字第51号,2014-01-10,a8dfddad-0113-49ac-9599-79fd9a8a6bfd,一审,被告人肖绿美犯受贿罪一审刑事判决书,http://wenshu.court.gov.cn/content/content?Doc...,2013,刑事
3,保山市隆阳区人民法院,（2018）云0502刑初456号,2018-09-17,77893be7-fbd2-4f3c-b84e-a97c00bcc773,一审,杨月龙诈骗一审刑事判决书,http://wenshu.court.gov.cn/content/content?Doc...,2018,刑事
4,上海市徐汇区人民法院,（2014）徐刑初字第1286号,2014-12-08,0f88d79a-fbac-4e5a-8262-87f0d8bc6d1b,一审,于雷峰盗窃案一审刑事判决书,http://wenshu.court.gov.cn/content/content?Doc...,2014,刑事


In [90]:
pageNum = 7
df.to_csv('./case_urls{}_8.csv'.format(pageNum), encoding='utf_8_sig')

### count case court locations

In [69]:
pd.DataFrame(df['casecourt'].str[:2]).groupby('casecourt').size()

casecourt
上海     7
云南    21
勐海     1
南丹     1
大丰     1
天津     1
山东     1
广东    10
广西     4
昆明     1
梧州     1
浙江     2
海南     1
湖南     2
甘肃     1
福建     1
腾冲     1
蕲春     1
陕西     1
黄山     1
dtype: int64